In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
! kaggle datasets download -d ifigotin/imagenetmini-1000

Dataset URL: https://www.kaggle.com/datasets/ifigotin/imagenetmini-1000
License(s): unknown
100% 3.91G/3.92G [00:56<00:00, 88.1MB/s]
100% 3.92G/3.92G [00:56<00:00, 74.4MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imagenetmini-1000.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

# Preprocess the datasets and apply the some featrue engineering to clean the garbage value from the dataset.

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [6]:
# Setting up parameters
batch_size = 128
epochs = 50
learning_rate = 0.1


In [7]:
# Step 1: Data Preparation and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [8]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_ds = train_datagen.flow_from_directory(
    '/content/imagenet-mini/train',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_ds = test_datagen.flow_from_directory(
    '/content/imagenet-mini/val',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 34745 images belonging to 1000 classes.
Found 3923 images belonging to 1000 classes.


In [9]:
# Step 2: Model Architecture Improvements
def build_cnn_model(input_shape=(128, 128, 3), num_classes=1000):
    inputs = Input(shape=input_shape)

    # Block 1
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)

    # Block 2
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)

    # Block 3
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.4)(x)

    # Fully connected layer with Global Average Pooling
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)  # L2 Regularization
    x = Dropout(0.5)(x)

    # Output layer
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model


In [10]:
train_ds.num_classes , validation_ds.num_classes

(1000, 1000)

In [11]:
# Instantiate the model
model = build_cnn_model(input_shape=(128, 128, 3), num_classes=train_ds.num_classes)


In [12]:
import keras

In [13]:
# Step 3: Compile the model with Adam optimizer and categorical cross-entropy loss
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
# Step 4: Callbacks for Advanced Training Techniques
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)


In [15]:
# Step 5: Model Training
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=validation_ds,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


247/272 ━━━━━━━━━━━━━━━━━━━━ 7:17 17s/step - accuracy: 0.0018 - loss: 8.0833

KeyboardInterrupt: 

In [ ]:
# Evaluate the performance metrics of the model...

test_loss , test_accuracy = model.evaluate(validation_ds)


In [ ]:
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Evaluate accuracy and loss on validation set
test_loss, test_accuracy = model.evaluate(validation_ds)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Step 2: Generate predictions and calculate confusion matrix
# Get the ground truth labels and predictions
true_labels = np.concatenate([y for x, y in validation_ds], axis=0)
pred_probs = model.predict(validation_ds)
pred_labels = np.argmax(pred_probs, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(np.argmax(true_labels, axis=1), pred_labels)

# Step 3: Visualize the Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=validation_ds.class_indices.keys(), yticklabels=validation_ds.class_indices.keys())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Step 4: Classification Report
print("\nClassification Report:")
print(classification_report(np.argmax(true_labels, axis=1), pred_labels, target_names=validation_ds.class_indices.keys()))


In [ ]:
# If the performance of the metrix is good than save the model and make the UI/UX on top if it.
# Save the model for future use case
# Optional: Save model for future use
model.save('cnn_model.h5')


In [9]:
# Split the data into the train and the validation split.

# generators
train_ds = keras.utils.image_dataset_from_directory(
    "/content/imagenet-mini/train",
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

validation_ds = keras.utils.image_dataset_from_directory(
    "/content/imagenet-mini/val",
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

Found 34745 files belonging to 1000 classes.
Found 3923 files belonging to 1000 classes.


In [20]:
type(train_ds)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [10]:
import numpy as np
import keras
from keras import layers

In [13]:
# Build the model..
input_shape = (256 , 256 , 3)
num_classes = 1000
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128 , kernel_size=(3 ,3) , activation="relu"),
        layers.MaxPooling2D(pool_size = (2 , 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1000)                │     115,201,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 115,294,248 (439.81 MB)

 Trainable params: 115,294,248 (439.81 MB)

 Non-trainable params: 0 (0.00 B)

In [1]:
# Compile the model
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training the model
model.fit(train_ds, epochs=epochs, validation_data=validation_ds, batch_size=batch_size)


NameError: name 'model' is not defined